In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=1a80ce55c34ae875cc23059e5d80c5e3c120a417cbacc62222e2c816cd353722
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [3]:
spark

#**RDD**

# Вспомогательные функции


In [9]:
path_to_order = "/content/drive/MyDrive/DE/LabRDD/dataset/order/order.csv"
path_to_customer = "/content/drive/MyDrive/DE/LabRDD/dataset/customer/customer.csv"
path_to_product = "/content/drive/MyDrive/DE/LabRDD/dataset/product/product.csv"

def createRDD(spark:SparkSession, path):
  return spark.sparkContext.textFile(path)\
                    .map(lambda x: x.split('\t'))

  # Задача 1
  * 1. Загрузить в RDD файл src/test/resources/input/order
  * 2. Выбрать строки в которых поле статус заказа delivered
  * 3. Выбрать ключ (customerID), значение (numberOfProduct, 1)
  * 4. Написать функцию reduce для расчёта
  * 5. Применит к RDD из п.3 метод reduceByKey c функцией из п.4
  * 6. Вывести или записать результат

In [27]:
orders = spark.sparkContext.textFile(path_to_order)
orders = orders.map(lambda x: x.split('\t'))\
              .filter(lambda x: 'delivered' in x[5])\
              .map(lambda x: (x[0], (x[3],1)))\
              .reduceByKey(lambda x,y: ((int(x[0])+int(y[0])),(x[1]+y[1]))).take(5)
print(orders)


[('1', (3000, 9)), ('4', (1300, 8)), ('2', (1250, 11)), ('3', ('100', 1)), ('5', (1300, 8))]


# Задача 2
   * 1. Загрузить в RDD файл src/test/resources/input/order
   * 2. Распарсить строки RDD
   * 3. Выбрать ключ поле (customerID), в значение (orderDate, numberOfProduct, productID)
   * 4. Загрузить в RDD файл customer
   * 5. Распарсить строки RDD
   * 6. Выбрать ключ поле (id), в значение (name)
   * 7. Выполнить внутреннее соединение RDD из п.6 и п.9
   * 8. Выбрать ключ (productID), в значение (customer.name, orderDate,  numberOfProduct)
   * 9. Загрузить в RDD файл product.csv
   * 10. Распарсить строки RDD
   * 11. Выбрать ключ (id), значение (price)
   * 12. Выполнить внутреннее соединение с RDD из п.11 и п.14
   * 13. Выбрать ключ (customer.name, order.orderDate), значение (order.numberOfProduct * product.price)
   * 14. Расчитать сумму в значении по ключу
   * 15. Вывести результат

# Задача 3
 * 1. Загрузить в RDD файл src/test/resources/input/product
 * 2. Распарсить строки в RDD
 * 3. Выбрать ключ поле id, в значение name
 * 4. Загрузить в RDD файд src/test/resources/input/order
 * 5. Распарсить строки в RDD
 * 6. Выбрать ключ поле productID, в значение numberOfProduct
 * 7. Посчитать кол-во проданных продуктов
 * 8. Выполнить левое соединение двух RDD
 * 9. Выполнить фильтрацию и оставить только те строки где значение numberOfProducts 0 или None
 * 10. Вывести результат или записать в директорию

# Задача 4
 * 1. Загрузить в RDD файд src/test/resources/input/order.csv
 * 2. Распарсить строки в RDD
 * 3. Выбрать только те транзакции у которых статус delivered
 * 4. Выбрать ключ (customerID), значение (numberOfProducts)
 * 5. Выполнить группировку по ключу
 * 6. Посчитать сумму по значению и разделить на размер коллекции
 * 7. Вывести результат или записать в директорию

# **DATAFRAME**

# Вспомогательные функции

In [48]:
from pyspark.sql.types import *
class Parameters:
  path_laptop = "/content/drive/MyDrive/DE/LabDF/dataset/laptop/laptop.txt"
  path_pc = "/content/drive/MyDrive/DE/LabDF/dataset/pc/pc.txt"
  path_printer = "/content/drive/MyDrive/DE/LabDF/dataset/printer/printer.txt"
  path_product = "/content/drive/MyDrive/DE/LabDF/dataset/product/product.txt"

  table_laptop = "laptop"
  table_pc = "pc"
  table_printer = "printer"
  table_product = "product"

  def createTable(self, name:str, spark:SparkSession, structType: StructType, path:str, delimiter: str = ','):
      spark.read\
            .options(delimiter = delimiter, nullValue = "\\N")\
            .schema(structType)\
            .csv(path)\
            .createOrReplaceTempView(name)
  def initTables(self, spark:SparkSession):
    self.createTable(Parameters.table_pc, spark, PC.structType, Parameters.path_pc)
    self.createTable(Parameters.table_product, spark, Product.structType, Parameters.path_product)

In [47]:
class PC:
  structType = StructType([
      StructField("ID", IntegerType()),
      StructField("MODEL", IntegerType()),
      StructField("SPEED", IntegerType()),
      StructField("RAM", IntegerType()),
      StructField("C2", IntegerType()),
      StructField("C3", StringType()),
      StructField("PRICE", IntegerType()),
  ])
class Product:
  structType = StructType([
      StructField("Maker", StringType()),
      StructField("Model", StringType()),
      StructField("Type", StringType())
  ])

In [49]:
p = Parameters()
p.initTables(spark)

# Задача 1
Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью.
Вывести: speed, средняя цена.

SELECT pc.speed, AVG(pc.price)
FROM pc
WHERE pc.speed > 600
GROUP BY pc.speed
)

In [38]:
from pyspark.sql.functions import col, avg
pc = spark.read.table("pc")\
              .select("speed","price")\
              .filter(col("price") > 600)\
              .groupBy("speed")\
              .agg(avg(col("price")).alias("avg_price"))\
              .select("speed","avg_price").show()

+-----+---------+
|speed|avg_price|
+-----+---------+
|  900|    980.0|
|  800|    970.0|
|  750|    900.0|
|  600|    850.0|
+-----+---------+



# Задача 2
Вывести все строки из таблицы Product, кроме трех строк с наименьшими номерами моделей и трех строк с наибольшими номерами моделей.

  Select maker, model, type from
  (
  Select
  row_number() over (order by model) p1,
  row_number() over (order by model DESC) p2,
  from Product
  ) t1
  where p1 > 3 and p2 > 3


In [56]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

product = spark.table("product")
MaxVal = Window.orderBy("model")
product1 = product.withColumn("p1", row_number().over(MaxVal))
MinVal = Window.orderBy(desc("model"))
product1.withColumn("p2", row_number().over(MinVal))\
        .select("maker", "model", "type")\
        .filter((col("p1")>3) & (col("p2")>3)).show()

+-----+-----+-------+
|maker|model|   type|
+-----+-----+-------+
|    B| 1750| Laptop|
|    E| 1434|Printer|
|    D| 1433|Printer|
|    A| 1408|Printer|
|    A| 1401|Printer|
|    C| 1321| Laptop|
|    A| 1298| Laptop|
|    D| 1288|Printer|
|    A| 1276|Printer|
|    E| 1260|     PC|
+-----+-----+-------+



# Задача 3
  Найти тех производителей ПК, все модели ПК которых имеются в таблице PC.

  SELECT p.maker
  FROM product p
  LEFT JOIN pc ON pc.model = p.model
  WHERE p.type = 'PC'
  GROUP BY p.maker
  HAVING COUNT(p.model) = COUNT(pc.model)
  

# Задача 4
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

  SELECT DISTINCT maker
  FROM product
   WHERE model IN (
  SELECT model
  FROM pc
  WHERE ram = (
    SELECT MIN(ram)
    FROM pc
    )
  AND speed = (
    SELECT MAX(speed)
    FROM pc
    WHERE ram = (
     SELECT MIN(ram)
     FROM pc
     )
    )
  )
  AND
  maker IN (
  SELECT maker
  FROM product
  WHERE type='printer'
  )

+-----+-----+-------+
|Maker|Model|   Type|
+-----+-----+-------+
|    B| 1121|     PC|
|    A| 1232|     PC|
|    A| 1233|     PC|
|    E| 1260|     PC|
|    A| 1276|Printer|
|    D| 1288|Printer|
|    A| 1298| Laptop|
|    C| 1321| Laptop|
|    A| 1401|Printer|
|    A| 1408|Printer|
|    D| 1433|Printer|
|    E| 1434|Printer|
|    B| 1750| Laptop|
|    A| 1752| Laptop|
|    E| 2113|     PC|
|    E| 2112|     PC|
+-----+-----+-------+

